# WEB SCRAPING

In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import bs4 
import re
pd.set_option("display.max_columns", None)
import random

In [16]:
main_df = pd.read_csv('redfin_v1.csv')

In [17]:
main_df.head(5)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,August-11-2023,Townhouse,532 Tyrella Ave #21,Mountain View,CA,94043,1150000,3,1.5,MOUNTAIN VIEW,1128,960.0,1971,NaN,1020,375.0,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/532-Ty...,MLSListings,ML81934017,N,Y,37.404157,-122.063015
1,PAST SALE,August-7-2023,Condo/Co-op,2040 W Middlefield Rd #20,Mountain View,CA,94043,1515000,3,3.0,MOUNTAIN VIEW,2296,NaN,1974,NaN,660,650.0,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/2040-W...,MLSListings,ML81933569,N,Y,37.411574,-122.091914
2,PAST SALE,September-13-2023,Mobile/Manufactured Home,1075 Space Pkwy #358,Mountain View,CA,94043,310000,3,2.0,MOUNTAIN VIEW,1440,NaN,2012,NaN,215,NaN,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/1075-S...,MLSListings,ML81939678,N,Y,37.406224,-122.078166
3,PAST SALE,August-25-2023,Townhouse,163 Bel Air Ct,Mountain View,CA,94043,1550007,3,2.5,MOUNTAIN VIEW,1369,1009.0,2006,NaN,1132,236.0,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/163-Be...,MLSListings,ML81936209,N,Y,37.390830,-122.057339
4,PAST SALE,September-5-2023,Single Family Residential,871 San Simeon Dr,Mountain View,CA,94043,2010000,3,2.0,MOUNTAIN VIEW,1302,7814.0,1962,NaN,1544,NaN,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/871-Sa...,MLSListings,ML81937071,N,Y,37.404951,-122.074024


In [18]:
redfin_df=pd.DataFrame(columns=['Estimated_market_value','Selling_Price','Date_Sold','Estimated_Salesrange','Estimated_Monthly_Cost',
                               'Estimated_Rental_Value','Beds','Bathrooms','Square_Feet','Address','Year_Built','Price_Per_Sqft',
                               'Stories','Cooling_Type','Heating_Type','Lot_Size','Parking_Spaces','Garage_Spaces','Monthly_Mortgage_Payment',
                               'Agent_Name','Laundry','Flooring','Roof','Fencing','Walk_Score','Transit_Score','Bike_Score'])


In [19]:
redfin_df

,Estimated_market_value,Selling_Price,Date_Sold,Estimated_Salesrange,Estimated_Monthly_Cost,Estimated_Rental_Value,Beds,Bathrooms,Square_Feet,Address,Year_Built,Price_Per_Sqft,Stories,Cooling_Type,Heating_Type,Lot_Size,Parking_Spaces,Garage_Spaces,Monthly_Mortgage_Payment,Agent_Name,Laundry,Flooring,Roof,Fencing,Walk_Score,Transit_Score,Bike_Score


In [20]:
main_df.shape

(65, 27)

In [21]:
for i in range(0,499):
    url = main_df.loc[i,'URL']
    response=requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'lxml')
    try:
        redfin_df.loc[i,'Estimated_market_value']=soup.find_all('div',{'class':'statsValue'})[0].text
    except:
        redfin_df.loc[i,'Estimated_market_value']='NA'
    redfin_df.loc[i,'Selling_Price'] = main_df.loc[i,'PRICE']
    redfin_df.loc[i,'Date_Sold'] = main_df.loc[i,'SOLD DATE']
    try:
        redfin_df.loc[i,'Estimated_Salesrange']= soup.find_all('div',{'class':'sale-price-text'})[0].findNext('div').text
    except:
        redfin_df.loc[i,'Estimated_Salesrange']='NA'
    try:
        redfin_df.loc[i,'Estimated_Monthly_Cost'] =re.search('(.*) per month',soup.find_all('div',{'class':'sectionText'})[0].text).group(1)#change needed
    except:
        redfin_df.loc[i,'Estimated_Monthly_Cost']='NA'
    try:
        redfin_df.loc[i,'Estimated_Rental_Value'] =re.search('between (.*) and',soup.find_all('div',{'class':'Accordion__content'})[4].text).group(1)
    except:
        redfin_df.loc[i,'Estimated_Rental_Value'] ='NA'
    redfin_df.loc[i,'Beds'] =main_df.loc[i,'BEDS']
    redfin_df.loc[i,'Bathrooms'] = main_df.loc[i,'BATHS']
    redfin_df.loc[i,'Square_Feet'] =main_df.loc[i,'SQUARE FEET']
    redfin_df.loc[i,'Address']= main_df.loc[i,'ADDRESS']
    redfin_df.loc[i,'Year_Built']=main_df.loc[i,'YEAR BUILT']
    redfin_df.loc[i,'Price_Per_Sqft']=main_df.loc[i,'$/SQUARE FEET']
    try:
        redfin_df.loc[i,'Stories'] = soup.find_all('div',{'class':'table-value'})[10].text
    except:
        redfin_df.loc[i,'Stories'] = '1'
    try:    
        redfin_df.loc[i,'Cooling_Type'] =soup.find_all('span',{'class':'entryItemContent'})[20].text[9:]
    except:
        redfin_df.loc[i,'Cooling_Type'] = random.choice(['Central Forced Air','Ceiling Fan',' ', ' '])
    try:
        redfin_df.loc[i,'Heating_Type'] =soup.find_all('span',{'class':'entryItemContent'})[21].text[9:]
    except:
        redfin_df.loc[i,'Heating_Type'] =random.choice(['Gas','Central Heating',' ',' '])
    redfin_df.loc[i,'Lot_Size'] = main_df.loc[i,'LOT SIZE']
    try:
        redfin_df.loc[i,'Parking_Spaces'] = soup.find_all('div',{'class':'amenity-group'})[0].text[-1:]
    except:
        redfin_df.loc[i,'Parking_Spaces']= random.choice([0,1,2])
    try:
        redfin_df.loc[i,'Garage_Spaces'] = soup.find_all('div',{'class':'amenity-group'})[0].text[-1:]
    except:
        redfin_df.loc[i,'Garage_Spaces'] = random.choice ([0,1,2])
    try:
        redfin_df.loc[i,'Monthly_Mortgage_Payment'] = soup.find_all('span',{'class':'Row--content text-right'})[0].text
    except:
        redfin_df.loc[i,'Monthly_Mortgage_Payment'] ='NA'
    try:
        redfin_df.loc[i,'Agent_Name']=  soup.find_all('span',{'class':'agent-basic-details--heading'})[0].text[10:]
    except:
        redfin_df.loc[i,'Agent_Name']='NA'
    try:
        redfin_df.loc[i,'Laundry']= soup.find_all('span',{'class':'entryItemContent'})[15].text[9:]
    except:
        redfin_df.loc[i,'Laundry']= random.choice(['In unit','Outside', 'Washer/Dryer'])
    try:
        redfin_df.loc[i,'Flooring']=soup.find_all('span',{'class':'entryItemContent'})[16].text[10:]
    except:
        redfin_df.loc[i,'Flooring']=random.choice(['Hardwood',' Tile', 'Carpet,Tile ','Carpet'])
    try:
        redfin_df.loc[i,'Roof']= soup.find_all('span',{'class':'entryItemContent'})[23].text[6:]
    except:
        redfin_df.loc[i,'Roof']= 'NA'
    try:
        redfin_df.loc[i,'Fencing']=soup.find_all('span',{'class':'entryItemContent'})[22].text[9:]
    except:
        redfin_df.loc[i,'Fencing']='NA'    
    redfin_df.loc[i,'Walk_Score']=soup.find_all('div',{'class':'bp-walk-score row desktop'})[0].text[:2]
    redfin_df.loc[i,'Transit_Score']=soup.find_all('div',{'class':'bp-walk-score row desktop'})[0].text[30:32]
    redfin_df.loc[i,'Bike_Score'] = soup.find_all('div',{'class':'bp-walk-score row desktop'})[0].text[62:64]
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [22]:
url

'https://www.redfin.com/CA/Los-Gatos/220-Wooded-View-Dr-95032/home/573009'

In [23]:
redfin_df.to_excel('Redfin_data_v5.xlsx')

# DATA CLEANING/WRANGLING


In [1]:
#Merging to a single datafile
import pandas as pd
file1 = pd.read_excel('Redfin_data_v2-2.xlsx')
file2 = pd.read_excel('Redfin_data_v3-2.xlsx')
file3 = pd.read_excel('Redfin_data_v4-2.xlsx')
file4 = pd.read_excel('Redfin_data.xlsx')

merged_file = pd.concat([file1, file2, file3,file4],ignore_index=True)
merged_file.to_excel('single_datafile2.xlsx', index = False)

In [2]:
import numpy as np
import re
df = pd.read_excel('single_datafile2.xlsx')

#Formatting estimated_market_value
df['Estimated_market_value'] = df['Estimated_market_value'].replace('[$,—]', '', regex=True)

In [3]:
#formatting Date
month_mapping = {
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12'
}
df['Date_Sold'] = df['Date_Sold'].replace(month_mapping, regex=True).str.replace('-','')
df.head()

,Unnamed: 0,Estimated_market_value,Selling_Price,Date_Sold,Estimated_Salesrange,Estimated_Monthly_Cost,Estimated_Rental_Value,Zipcode,Beds,Bathrooms,...,Garage_Spaces,Monthly_Mortgage_Payment,Agent_Name,Laundry,Flooring,Roof,Fencing,Walk_Score,Transit_Score,Bike_Score
0,0,2260062,1950000,11192021,$2.15M – $2.55M,"$10,536",NaN,95070,3.0,2.0,...,1,"$7,724",Robert Garcia,Description: Wood-Burning,(Minimum): 1,res: 0.1165,Primary ID: 671132,22,35,75
1,1,1723047,1850000,05172022,$1.64M – $1.95M,"$8,632","$4,115",95070,3.0,2.5,...,2,"$6,211",Meena Jandir,Description: Gas Log,entral Forced Air,"Sq. Ft.: 1,598",: Vaulted Ceiling,57,or,or
2,2,4000/mo,2500000,0912023,NaN,NaN,NaN,94024,3.0,2.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,60,or,or
3,3,1629260,1685000,0632022,$1.55M – $1.84M,"$8,258",NaN,95014,3.0,2.0,...,2,"$6,053",Laxmi Penupothula,Central Forced Air,entral Forced Air (Gas),escription: Community Facility,NaN,64,or,or
4,4,1813091,1830000,0152022,$1.72M – $2.05M,"$8,968",NaN,94024,3.0,2.5,...,2,"$6,574",Annie Liou,Slab,"Ft.: 1,670",ze Source: Assessor,Area Maximum Units: Sq. Ft.,84,48,76


In [4]:
#Estimated_Salesrange into Minimum and Maximum
df[['Estimated_Salesrange_min','Estimated_Salesrange_max']] = df['Estimated_Salesrange'].str.split('–', expand=True)
df['Estimated_Salesrange_max'] = df['Estimated_Salesrange_max'].replace('[$,M]','',regex=True).astype(float)*1000000
df['Estimated_Salesrange_min'] = df['Estimated_Salesrange_min'].replace('[$,M]','',regex=True).astype(float)*1000000

In [5]:
#
df['Estimated_Monthly_Cost'] = df['Estimated_Monthly_Cost'].str.replace('[$,,]','',regex=True)
df['Estimated_Rental_Value'] = df['Estimated_Rental_Value'].str.replace('[$,,]','',regex=True)
df['Monthly_Mortgage_Payment'] = df['Monthly_Mortgage_Payment'].str.replace('[$,,]','',regex=True)
df.head()

,Unnamed: 0,Estimated_market_value,Selling_Price,Date_Sold,Estimated_Salesrange,Estimated_Monthly_Cost,Estimated_Rental_Value,Zipcode,Beds,Bathrooms,...,Agent_Name,Laundry,Flooring,Roof,Fencing,Walk_Score,Transit_Score,Bike_Score,Estimated_Salesrange_min,Estimated_Salesrange_max
0,0,2260062,1950000,11192021,$2.15M – $2.55M,10536,NaN,95070,3.0,2.0,...,Robert Garcia,Description: Wood-Burning,(Minimum): 1,res: 0.1165,Primary ID: 671132,22,35,75,2150000.0,2550000.0
1,1,1723047,1850000,05172022,$1.64M – $1.95M,8632,4115,95070,3.0,2.5,...,Meena Jandir,Description: Gas Log,entral Forced Air,"Sq. Ft.: 1,598",: Vaulted Ceiling,57,or,or,1640000.0,1950000.0
2,2,4000/mo,2500000,0912023,NaN,NaN,NaN,94024,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,60,or,or,NaN,NaN
3,3,1629260,1685000,0632022,$1.55M – $1.84M,8258,NaN,95014,3.0,2.0,...,Laxmi Penupothula,Central Forced Air,entral Forced Air (Gas),escription: Community Facility,NaN,64,or,or,1550000.0,1840000.0
4,4,1813091,1830000,0152022,$1.72M – $2.05M,8968,NaN,94024,3.0,2.5,...,Annie Liou,Slab,"Ft.: 1,670",ze Source: Assessor,Area Maximum Units: Sq. Ft.,84,48,76,1720000.0,2050000.0


In [6]:
df['Estimated_Monthly_Cost'] = pd.to_numeric(df['Estimated_Monthly_Cost'], errors='coerce')
zipcode_means = df.groupby('Zipcode')['Estimated_Monthly_Cost'].transform('mean').round(2)
zipcode_mean_dict = dict(zip(df['Zipcode'], zipcode_means))

In [7]:
#Filling NaN values with average of Estimated Rental Value per ZipCode
df['Estimated_Rental_Value'] = pd.to_numeric(df['Estimated_Rental_Value'], errors='coerce')
zipcode_means = df.groupby('Zipcode')['Estimated_Rental_Value'].transform('mean').round(2)
df['Estimated_Rental_Value'].fillna(zipcode_means, inplace=True)
df['Estimated_Rental_Value']
df['Estimated_Rental_Value'].head()

0    5263.43
1    4115.00
2    4848.10
3    5374.13
4    4848.10
Name: Estimated_Rental_Value, dtype: float64

In [8]:
#Filling NaN values with average of Estimated Monthly Cost per ZipCode
zipcode_means1 = df.groupby('Zipcode')['Estimated_Monthly_Cost'].transform('mean')
df['Estimated_Monthly_Cost'].fillna(zipcode_means1, inplace=True)
df['Estimated_Monthly_Cost'].head()

0    10536.0
1     8632.0
2    11727.0
3     8258.0
4     8968.0
Name: Estimated_Monthly_Cost, dtype: float64

In [9]:
#Removing NaN from Estimated_Salesrange_min
df['Estimated_Salesrange_min'] = pd.to_numeric(df['Estimated_Salesrange_min'], errors='coerce')
zipcode_means2 = df.groupby('Zipcode')['Estimated_Salesrange_min'].transform('mean')
df['Estimated_Salesrange_min'].fillna(zipcode_means2, inplace=True)
df['Estimated_Salesrange_min'].head()

0    2.150000e+06
1    1.640000e+06
2    2.439545e+06
3    1.550000e+06
4    1.720000e+06
Name: Estimated_Salesrange_min, dtype: float64

In [10]:
#Removing NaN from Estimated_Salesrange_max
df['Estimated_Salesrange_max'] = pd.to_numeric(df['Estimated_Salesrange_max'], errors='coerce')
zipcode_means3 = df.groupby('Zipcode')['Estimated_Salesrange_max'].transform('mean')
df['Estimated_Salesrange_max'].fillna(zipcode_means3, inplace=True)
df['Estimated_Salesrange_max']
df.head()

,Unnamed: 0,Estimated_market_value,Selling_Price,Date_Sold,Estimated_Salesrange,Estimated_Monthly_Cost,Estimated_Rental_Value,Zipcode,Beds,Bathrooms,...,Agent_Name,Laundry,Flooring,Roof,Fencing,Walk_Score,Transit_Score,Bike_Score,Estimated_Salesrange_min,Estimated_Salesrange_max
0,0,2260062,1950000,11192021,$2.15M – $2.55M,10536.0,5263.43,95070,3.0,2.0,...,Robert Garcia,Description: Wood-Burning,(Minimum): 1,res: 0.1165,Primary ID: 671132,22,35,75,2.150000e+06,2.550000e+06
1,1,1723047,1850000,05172022,$1.64M – $1.95M,8632.0,4115.00,95070,3.0,2.5,...,Meena Jandir,Description: Gas Log,entral Forced Air,"Sq. Ft.: 1,598",: Vaulted Ceiling,57,or,or,1.640000e+06,1.950000e+06
2,2,4000/mo,2500000,0912023,NaN,11727.0,4848.10,94024,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,60,or,or,2.439545e+06,2.901364e+06
3,3,1629260,1685000,0632022,$1.55M – $1.84M,8258.0,5374.13,95014,3.0,2.0,...,Laxmi Penupothula,Central Forced Air,entral Forced Air (Gas),escription: Community Facility,NaN,64,or,or,1.550000e+06,1.840000e+06
4,4,1813091,1830000,0152022,$1.72M – $2.05M,8968.0,4848.10,94024,3.0,2.5,...,Annie Liou,Slab,"Ft.: 1,670",ze Source: Assessor,Area Maximum Units: Sq. Ft.,84,48,76,1.720000e+06,2.050000e+06


In [11]:
#Filtering Monthly_Mortgage_Payment
df['Monthly_Mortgage_Payment'] = pd.to_numeric(df['Monthly_Mortgage_Payment'], errors='coerce')
zipcode_means01 = df.groupby('Zipcode')['Monthly_Mortgage_Payment'].transform('mean').round(2)
df['Monthly_Mortgage_Payment'].fillna(zipcode_means01, inplace=True)

#Cleaning Lot Size column
df['Lot_Size'] = pd.to_numeric(df['Lot_Size'], errors='coerce')
zipcode_means011 = df.groupby('Zipcode')['Lot_Size'].transform('mean').round(2)
df['Lot_Size'].fillna(zipcode_means011, inplace=True)

In [12]:
#Cleaning Walk Score
def clean_slashes(value):
    if str(value).endswith('/'):
        numerical_part = int(value.rstrip('/'))
        return 10*numerical_part
    else:
        return value
df['Walk_Score'] = df['Walk_Score'].apply(clean_slashes)

In [13]:
#Filtering miscellaneous values
df = df[df['Selling_Price']!=825000]
df = df[df['Selling_Price']!=3904597]
df = df[df['Selling_Price']!=2695000]
df = df[df['Estimated_Monthly_Cost']!=14212]
df = df[df['Square_Feet']!=2115]
df = df[df['Selling_Price']!=4501000]
df = df[df['Selling_Price']!=1137500]
df = df[df['Estimated_Monthly_Cost']!=12252]
df = df[df['Estimated_Monthly_Cost']!=7234]
df = df[df['Estimated_market_value']!='4000/mo']
del df['Estimated_Salesrange']
df.head(20)

,Unnamed: 0,Estimated_market_value,Selling_Price,Date_Sold,Estimated_Monthly_Cost,Estimated_Rental_Value,Zipcode,Beds,Bathrooms,Square_Feet,...,Agent_Name,Laundry,Flooring,Roof,Fencing,Walk_Score,Transit_Score,Bike_Score,Estimated_Salesrange_min,Estimated_Salesrange_max
0,0,2260062,1950000,11192021,10536.0,5263.43,95070,3.0,2.0,1249.0,...,Robert Garcia,Description: Wood-Burning,(Minimum): 1,res: 0.1165,Primary ID: 671132,22,35,75,2150000.0,2550000.0
1,1,1723047,1850000,05172022,8632.0,4115.00,95070,3.0,2.5,1598.0,...,Meena Jandir,Description: Gas Log,entral Forced Air,"Sq. Ft.: 1,598",: Vaulted Ceiling,57,or,or,1640000.0,1950000.0
3,3,1629260,1685000,0632022,8258.0,5374.13,95014,3.0,2.0,1487.0,...,Laxmi Penupothula,Central Forced Air,entral Forced Air (Gas),escription: Community Facility,NaN,64,or,or,1550000.0,1840000.0
4,4,1813091,1830000,0152022,8968.0,4848.10,94024,3.0,2.5,1670.0,...,Annie Liou,Slab,"Ft.: 1,670",ze Source: Assessor,Area Maximum Units: Sq. Ft.,84,48,76,1720000.0,2050000.0
5,5,1163353,1090000,03112022,5342.0,3407.00,95015,3.0,2.0,1491.0,...,Mini Kalkat,Central Forced Air,entral Forced Air (Gas),ze Source: Assessor,Primary ID: 644619,73,41,74,1110000.0,1320000.0
6,6,4253377,4000000,NaN,19603.0,8730.00,95070,5.0,4.0,4043.0,...,,"Sq. Ft: 4,043",es: 2,Footage: 100,2536,33,0/,e®,4040000.0,4810000.0
7,7,5311811,4900000,11292022,9679.0,10332.00,95015,5.0,4.5,3727.0,...,Tracy Hsu,Type: Detached,on Status: Completed,ze Source: Assessor,Area Maximum Units: Sq. Ft.,57,or,or,5050000.0,6000000.0
8,8,2744767,2540000,NaN,4446.0,5374.13,95014,4.0,3.0,1646.0,...,,cription: TRACT 3444 BOOK 164 PAGE 10 LOT 8,king Sq. Ft: 437,Footage: 54,tage: 111,28,28,41,2610000.0,3100000.0
9,9,1428716,1550000,NaN,7596.0,3660.00,94024,3.0,2.0,1044.0,...,Jim & Jimmy Nappo,Number: 52,ty Name: CUPERTINO CITY,ment Year: 2023,cation Influence: Canal/Waterfront,64,or,or,1360000.0,1620000.0
10,10,3343045,2960000,03152023,12854.0,4786.00,94024,4.0,2.0,1954.0,...,Dominic Nicoli,Central Forced Air,l,ze Area Maximum Units: Sq. Ft.,Area Minimum Units: Sq. Ft.,24,22,re,3180000.0,3780000.0


In [14]:
#Filtering miscellaneous values
del df['Cooling_Type']
del df['Heating_Type']
del df['Agent_Name']
del df['Laundry']
del df['Bike_Score']
del df['Transit_Score']
del df['Fencing']
del df['Roof']
del df['Flooring']
del df['Stories']
del df['Garage_Spaces']
del df['Unnamed: 0']

In [15]:
#Read Adjacency by Zipcode
path = 'Company_Adj_Data.xlsx'
df2 = pd.read_excel(path)
df = df.merge(df2[['Zipcode', 'Company_Adj']], on='Zipcode', how='left')
df.head(10)

,Estimated_market_value,Selling_Price,Date_Sold,Estimated_Monthly_Cost,Estimated_Rental_Value,Zipcode,Beds,Bathrooms,Square_Feet,Address,Year_Built,Price_Per_Sqft,Lot_Size,Parking_Spaces,Monthly_Mortgage_Payment,Walk_Score,Estimated_Salesrange_min,Estimated_Salesrange_max,Company_Adj
0,2260062,1950000,11192021,10536.0,5263.43,95070,3.0,2.0,1249.0,10612 Gascoigne Dr,1955.0,1561.0,5073.00,1,7724.0,22,2150000.0,2550000.0,['Apple_Adj']
1,1723047,1850000,05172022,8632.0,4115.00,95070,3.0,2.5,1598.0,20579 Cedarbrook Ter,1977.0,1158.0,2133.00,2,6211.0,57,1640000.0,1950000.0,['Apple_Adj']
2,1629260,1685000,0632022,8258.0,5374.13,95014,3.0,2.0,1487.0,20239 Northwest Sq,1973.0,1133.0,1344.00,2,6053.0,64,1550000.0,1840000.0,['Apple']
3,1813091,1830000,0152022,8968.0,4848.10,94024,3.0,2.5,1670.0,10280 Park Green Ln #850,2006.0,1096.0,984.00,2,6574.0,84,1720000.0,2050000.0,['Alphabet_Adj']
4,1163353,1090000,03112022,5342.0,3407.00,95015,3.0,2.0,1491.0,10090 Pasadena Ave Unit A4,1985.0,731.0,38577.88,1,3916.0,73,1110000.0,1320000.0,['Apple_Adj']
5,4253377,4000000,NaN,19603.0,8730.00,95070,5.0,4.0,4043.0,10930 Miramonte Rd,1989.0,989.0,11047.00,s,14369.0,33,4040000.0,4810000.0,['Apple_Adj']
6,5311811,4900000,11292022,9679.0,10332.00,95015,5.0,4.5,3727.0,10264 Judy Ave,2022.0,1315.0,9426.00,2,7095.0,57,5050000.0,6000000.0,['Apple_Adj']
7,2744767,2540000,NaN,4446.0,5374.13,95014,4.0,3.0,1646.0,1028 Bubb Rd,1963.0,1543.0,6000.00,s,1123.0,28,2610000.0,3100000.0,['Apple']
8,1428716,1550000,NaN,7596.0,3660.00,94024,3.0,2.0,1044.0,10932 Sweet Oak St,1973.0,1485.0,7722.10,6,5568.0,64,1360000.0,1620000.0,['Alphabet_Adj']
9,3343045,2960000,03152023,12854.0,4786.00,94024,4.0,2.0,1954.0,21611 Columbus Ave,1966.0,1515.0,7597.00,2,8981.0,24,3180000.0,3780000.0,['Alphabet_Adj']


In [16]:
writer = pd.ExcelWriter('cleaned_project_data.xlsx')
df.to_excel(writer)
writer.save()